## Module with neural network class.

The class is designed following "Hands On Machine Learning" by A.Geron

In [1]:
import tensorflow as tf 
import numpy as np
import math
import h5py
import random
import matplotlib.pyplot as plt

In [2]:
class Network:

    def __init__(self,layers,batch_size,fl_name,regression=False):
        self.regression=regression
        if len(layers)<3:
            raise ValueError
        self.batch_size=batch_size
        self.layers=layers
        self.X=tf.placeholder(tf.float32,shape=(None,layers[0]),name="X")
        if regression:
            self.y=tf.placeholder(tf.float32,shape=(None),name="y")
        else:
            self.y=tf.placeholder(tf.int64,shape=(None),name="y")
        self.activations={"ELU":tf.nn.elu,"RELU":tf.nn.relu,"SIG":tf.nn.sigmoid,"TANH":tf.nn.tanh,"SOFT":tf.nn.softplus}

        self.tr_err=[]
        self.val_err=[]
        self.filename="./"+fl_name+".ckpt"
    
    def create_network(self,learning_rate,activ="ELU",optim="SGD",initialization="HE"):
        with tf.name_scope("dnn"):
            if initialization=="HE":
                he_init=tf.contrib.layers.variance_scaling_initializer()
                self.H=[tf.layers.dense(self.X,self.layers[1],name="Hidden_layer1",kernel_initializer=he_init,activation=self.activations[activ])]
            else:
                self.H=[tf.layers.dense(self.X,self.layers[1],name="Hidden_layer1",activation=self.activations[activ])]

            if len(self.layers)>3:
                for i in range(2,len(self.layers)):
                    self.H.append(tf.layers.dense(self.H[-1],self.layers[i],name="Hidden_layer"+str(i),activation=self.activations[activ]))

            self.logits=tf.layers.dense(self.H[-1],self.layers[-1],name="Output_layer")

        with tf.name_scope("loss"):
            if self.regression:
                lsq=(tf.transpose(self.logits)-self.y)**2
                self.loss=tf.reduce_mean(lsq,name="Loss")
            else:
                xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.y,logits=self.logits)
                self.loss=tf.reduce_mean(xentropy,name="Loss")

        with tf.name_scope("train"):
            if optim=="SGD":
                optimizer=tf.train.GradientDescentOptimizer(learning_rate)
            elif optim=="MOM":
                optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
            elif optim=="NMOM":
                optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9,use_nesterov=True)
            elif optim=="RMSP":
                optimizer=tf.train.RMSPropOptimizer(learinng_rate=learning_rate,momentum=0.9,decay=0.9,epsilon=1e-10)
            elif optim=="ADAM":
                optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)

            self.training_op=optimizer.minimize(self.loss)


        with tf.name_scope("eval"):
            if self.regression:
                correct=np.absolute(tf.transpose(self.logits)-self.y)
                self.accuracy=tf.reduce_mean(correct)
    
            else:
                correct=tf.nn.in_top_k(self.logits,self.y,1)
                self.accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))



    def train(self,Data_train,Labels_train,Data_eval,Labels_eval,n_epochs,save_model=False):

        self.indices=np.arange(Data_train.shape[0])
        
        init=tf.global_variables_initializer()
        
        if save_model:
            saver=tf.train.Saver()

        with tf.Session() as sess:
            init.run()

            for epoch in range(n_epochs):
                
                np.random.shuffle(self.indices)
                
                for i in range(len(Data_train)//batch_size):
                    X_batch,y_batch=self.get_batch(Data_train,Labels_train,i)
                    sess.run(self.training_op,feed_dict={self.X:X_batch,self.y:y_batch})

                acc_train=self.accuracy.eval(feed_dict={self.X:Data_train,self.y:Labels_train})
                acc_val=self.accuracy.eval(feed_dict={self.X:Data_eval,self.y:Labels_eval})
                self.tr_err.append(acc_train)
                self.val_err.append(acc_val)

                print(epoch, "Train accuracy:", acc_train, "Val accuracy:",acc_val)

            if save_model:
                save_path=saver.save(sess,self.filename)


    def get_batch(self,X,y,run):

        inds=self.indices[self.batch_size*run:self.batch_size*(run+1)]
        BX=[]
        By=[]
        for i in inds:
            BX.append(X[i])
            By.append(y[i])
        return np.array(BX),np.array(By)

    def plot_error(self):

        fig=plt.figure()
        ax1=fig.add_subplot(111)
        ax1.plot(self.tr_err,label="Training")
        ax1.plot(self.val_err,label="Validation")
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Error')
        ax1.legend()
        plt.show()

    def predict(self,Data):

        with tf.Session() as sess:
            saver=tf.train.Saver()
            saver.restore(sess,self.filename)
            res=self.logits.eval(feed_dict={self.X:Data})
            if self.regression:
                return res
            else:
                return np.argmax(res,axis=1)
            
    def score(self,Data,Labels):
        with tf.Session() as sess:
            saver=tf.train.Saver()
            saver.restore(sess,self.filename)
            return self.accuracy.eval(feed_dict={self.X:Data,self.y:Labels})
         